In [19]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [20]:
model = keras.models.load_model('./Trained Models/m1.h5')
url = './Testing/test2.mp4'
# url = '/home/avishrant/Downloads/test.mp4'

In [21]:
import cv2
class Draw(object):
    def __init__(self):
        pass

    def set_ref(self, frame):
        self.ref = frame
    
    def drawfunc(self, frame):
        diff = cv2.absdiff(self.ref, frame)
        gray = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY)
        blur = cv2.GaussianBlur(gray, (5,5), 0)
        _, thresh = cv2.threshold(blur, 20, 255, cv2.THRESH_BINARY)
        dilated = cv2.dilate(thresh, None, iterations=3)
        contours, _ = cv2.findContours(dilated, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

        self.ref = frame.copy()
        for contour in contours:
            (x, y, w, h) = cv2.boundingRect(contour)
            if cv2.contourArea(contour) < 1500:
                continue
            frame = cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        return frame

In [28]:
# Live Generator
vid = cv2.VideoCapture(url)
font = cv2.FONT_HERSHEY_SIMPLEX 
org = (10, 40) 
fontScale = 0.7
thickness = 1
f_stat = False
obj = Draw()
while (cv2.waitKey(1) == -1):
    ret, frame = vid.read()
    
    if not ret:
        break
    tmp = cv2.resize(frame , (150, 150))
    tmp = tmp / 255.0
    pred = model.predict(np.array([tmp]))
    if not f_stat:
        f_stat = True
        obj.set_ref(frame) 
    else:
        frame = obj.drawfunc(frame)

    string = "Suspicious" if pred[0][0] < 0.5 else "Peaceful"
    string += f" {str(pred[0][0])}"
    color = (0, 0, 255) if pred[0][0] < 0.5 else (255, 0, 0)
    frame = cv2.putText(frame, string, org, font, fontScale, color, thickness, cv2.LINE_AA) 
    cv2.imshow("Video", frame)
    
vid.release()
cv2.destroyAllWindows()

In [15]:
# Image Generator      
import cv2
import numpy as np
vid = cv2.VideoCapture(url)
lst = []
while (cv2.waitKey(1) == -1):
    ret, frame = vid.read()
    if not ret:
        break
    lst.append(cv2.resize(frame, (64,64)))

In [81]:
lst2 = np.array(lst)
pred = model.predict(lst2)

In [82]:
output_folder = 'output/'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

In [83]:
count = 0
fcount = 0
vid = cv2.VideoCapture(url)
for i in range(len(pred)):
    ret, frame = vid.read()
    if fcount == i and pred[i] < 0.1:
        face_filename = '%s/%d.jpg' % (output_folder, count)
        cv2.imwrite(face_filename, frame)
        count += 1
    fcount += 1
    